In [ ]:
# dataset: http://www.manythings.org/anki/deu-eng.zip
import string
import re
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Função que abre um arquivo de texto 
def abre_txt(endereco_arquivo):
    arquivo = open(endereco_arquivo, mode='rt', encoding='utf-8') # mode='rt' é para abrir o arquivo no modo leitura
    texto = arquivo.read()
    arquivo.close()
    return texto

In [ ]:
# Função que quebra o texto em uma lista de palavras
def lista_palavras(texto):
    lista = texto.strip().split('\n')
    lista = [i.split('\t') for i in lista]
    return lista

In [ ]:
dataset = abre_txt("C:/Users/Natan/OneDrive/Documentos/DidaticaTech/deu.txt")
alemao_ingles = lista_palavras(dataset)
alemao_ingles = array(alemao_ingles)

In [ ]:
# Pegando apenas uma parte do dataset, para poupar processamento:
alemao_ingles = alemao_ingles[:50000,:]

In [ ]:
alemao_ingles[24000]

In [ ]:
cont=0
for i in alemao_ingles[:,0]:
    print(i)
    cont+=1
    if cont>5:
        break

In [ ]:
# Removendo pontuações:
alemao_ingles[:,0] = [re.sub('[^\w\s]','',s) for s in alemao_ingles[:,0]]
alemao_ingles[:,1] = [re.sub('[^\w\s]','',s) for s in alemao_ingles[:,1]]

In [ ]:
alemao_ingles

In [ ]:
# Transformando tudo para minúsculas
for i in range(len(alemao_ingles)):
    alemao_ingles[i,0] = alemao_ingles[i,0].lower()
    alemao_ingles[i,1] = alemao_ingles[i,1].lower()

In [ ]:
# Visualizando o tamanho das sentenças em cada idioma:

ingles = []
alemao = []

for i in alemao_ingles[:,0]:
    ingles.append(len(i.split()))

for i in alemao_ingles[:,1]:
    alemao.append(len(i.split()))

df = pd.DataFrame({'ingles':ingles, 'alemao':alemao})

tamanho_max_ingles = df['ingles'].max()
tamanho_max_alemao = df['alemao'].max()

print('Máximo comprimento inglês:', tamanho_max_ingles)
print('Máximo comprimento alemão:', tamanho_max_alemao)

# Visualizando um histograma de cada idioma:
df.hist(bins = 30)
plt.show()

In [ ]:
# Função que tokeniza as palavras:
def tokenizador(frases):
    tokenizador = Tokenizer()
    tokenizador.fit_on_texts(frases)
    return tokenizador

In [ ]:
# Tokenizador inglês
tokenizador_ingles = tokenizador(alemao_ingles[:, 0])
tamanho_vocabulario_ingles = len(tokenizador_ingles.word_index) + 1

print('Tamanho do vocabulário inglês:', tamanho_vocabulario_ingles)

In [ ]:
# Tokenizador alemão
tokenizador_alemao = tokenizador(alemao_ingles[:, 1])
tamanho_vocabulario_alemao = len(tokenizador_alemao.word_index) + 1

print('Tamanho do vocabulário alemão:', tamanho_vocabulario_alemao)

In [ ]:
# A tokenização já foi preparada para o dataset inteiro, agora só falta aplicar ela em cada subgrupo de treino e teste.
# Primeiro vamos dividir o dataset entre treino e teste, depois vamos tokenizar cada idioma, em cada grupo (treino e teste). 
# Ao fazer isso, pode ser que o comprimento máximo das sentenças do treino inglês fique diferente do seu teste, afinal esse 
# comprimento será definido a partir do tamanho máximo encontrado no conjunto que estiver sendo tokenizado naquele momento. 
# Por isso, iremos usar o parâmetro maxlen dentro de pad_sequences, que irá determinar o tamanho máximo independentemente do 
# tamanho da máxima sentença no dataset que estiver sendo informado. 
# Nesse caso vou usar o tamanho máximo total inglês para o dataset inglês e vou usar o máximo do alemão com o dataset alemão. 
# Obviamente, muitas amostras ficarão com valores zero em algumas entradas. Posso escolher se quero deixar esses zeros
# à esquerda ou à direita da sentença. Deixar os zeros à direita é melhor porque assim a frase sempre vai começar no mesmo ponto,
# e quando chegarem os zeros é sinal de que a frase terminou. Para definir isso, basta usar o parâmetro padding='post' em vez de
# padding = 'pre' que é o default.

def encoder_frases(tokenizador, comprimento, frases):
    frases_tokenizadas = tokenizador.texts_to_sequences(frases)
    frases_tokenizadas = pad_sequences(frases_tokenizadas, maxlen=comprimento, padding='post')
    return frases_tokenizadas

In [ ]:
from sklearn.model_selection import train_test_split

treino, teste = train_test_split(alemao_ingles, test_size=0.2)

In [ ]:
# Colocando os textos em alemão como variáveis preditoras, e os textos em inglês como variáveis target:
x_treino = encoder_frases(tokenizador_alemao, tamanho_max_alemao, treino[:, 1])
y_treino = encoder_frases(tokenizador_ingles, tamanho_max_ingles, treino[:, 0])

x_teste = encoder_frases(tokenizador_alemao, tamanho_max_alemao, teste[:, 1])
y_teste = encoder_frases(tokenizador_ingles, tamanho_max_ingles, teste[:, 0])

In [ ]:
x_treino.shape

In [ ]:
y_treino.shape

In [ ]:
# Criando o modelo de tradução com LSTMs:
modelo = Sequential()
modelo.add(Embedding(input_dim=tamanho_vocabulario_alemao, output_dim=500, input_length=tamanho_max_alemao))
modelo.add(LSTM(500))
modelo.add(RepeatVector(tamanho_max_ingles))
modelo.add(LSTM(500, return_sequences=True))
modelo.add(Dense(tamanho_vocabulario_ingles, activation='softmax'))

otimizador = optimizers.RMSprop(lr=0.001)
modelo.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

# A função de custo sparse_categorical_crossentropy é semelhante à categorical_crossentropy, mas ela permite que a variável
# target seja utilizada da forma como colocamos, sem precisar estar no formato one-hot-encoding. Isso ajuda a otimizar memória, 
# já que o vocabulário de saída é muito grande. 

In [ ]:
historico = modelo.fit(x_treino, y_treino.reshape(y_treino.shape[0], y_treino.shape[1], 1), epochs=40, batch_size=500, 
                       validation_split = 0.2, verbose=1)
# Obs: y_treino precisa estar no formado (amostras, timesteps, features)

In [ ]:
# Mostrando a evolução do treinamento:
plt.plot(historico.history['loss'])
plt.plot(historico.history['val_loss'])
plt.legend(['treino','teste'])
plt.show()

In [ ]:
# Fazendo predições:
previsoes = modelo.predict_classes(x_teste.reshape((x_teste.shape[0],x_teste.shape[1])))

In [ ]:
previsoes

In [ ]:
# Criando uma função que retorna as palavras a partir de seus números de tokenização:
def coleta_palavra(token, tokenizador):
    for palavra, index in tokenizador.word_index.items():
        if index == token:
            return palavra
    return None

# Transformando todas as predições em palavras:
texto_previsto = []
for frase in previsoes: # para cada frase prevista
    sentenca = []
    for token in range(len(frase)): # para cada token previsto dentro de uma frase
        palavra_ingles = coleta_palavra(frase[token], tokenizador_ingles) # palavra inglês
        if(palavra_ingles == None):
            sentenca.append('')
        else:
            sentenca.append(palavra_ingles)

    texto_previsto.append(' '.join(sentenca))

In [ ]:
# Criando um dataset que contém as frases em inglês do dataset de teste e as respectivas previsões do modelo:
df_previsoes = pd.DataFrame({'referências' : teste[:,0], 'previsões' : texto_previsto})

In [ ]:
# Visualizando alguns resultados:
df_previsoes

In [ ]:
# Mostrando aleatoriamente 10 frases do dataset:
df_previsoes.sample(10)

In [ ]:
# pip install nltk
# Aprendendo a calcular o BLEU SCORE em uma sentença:
from nltk.translate.bleu_score import sentence_bleu # usado para calcular uma sentença
referencias = [['ele', 'leu', 'o', 'texto'], ['ele', 'leu', 'um', 'livro']]
traducao = ['ele', 'leu', 'o', 'artigo']

score = sentence_bleu(referencias, traducao, weights=(0, 0, 0, 1))
print(score)
# Se quero avaliar uma palavra por vez, devo passar o peso 1 no primeiro item (1, 0, 0, 0). Se quero que avalie por grupos de 2
# palavras (bigram), preciso passar o peso 1 no segundo item (0, 1, 0, 0), e assim por diante. Posso avaliar até 4 palavras em 
# sequência (4-gram).
# Se quiser, posso considerar a avaliação de vários n-grams ao mesmo tempo, passando o peso de 0.25 para os 4, ou 0.33 para três
# deles, etc.

In [ ]:
# Aplicando o BLEU SCORE em várias sentenças:
from nltk.translate.bleu_score import corpus_bleu # usado para calcular várias sentenças

referencias = [[['ele', 'leu', 'o', 'texto'], ['ele', 'leu', 'um', 'livro']], [['ela', 'joga', 'xadrez']]]
candidatos = [['ele', 'leu', 'o', 'artigo'], ['ela', 'quer', 'um', 'lanche']]

score = corpus_bleu(referencias, candidatos, weights=(0.25, 0.25, 0.25, 0.25))
print(score)

# Repare que preciso informar listas dentro de listas. Nos candidatos, cada lista será comparada com sua respectiva referência, 
# por isso há várias listas de candidatos. 
# Nas referências, existe um agrupamento a mais porque podemos ter mais de uma referência para o mesmo candidato. Mas a lógica
# é a mesma, temos uma lista de referências para cada candidato.

In [ ]:
# Transformando nossos resultados em listas para aplicar o corpus_bleu:
candidatos = []
for i in df_previsoes['previsões']:
    candidatos.append(i.split()) # i.split() irá quebrar cada frase em uma lista de palavras individuais. Essa lista será colocada dentro de outra lista (candidatos)

referencias = []
for i in df_previsoes['referências']:
    lista = [i.split()]
    referencias.append(lista)

In [ ]:
referencias

In [ ]:
score = corpus_bleu(referencias, candidatos, weights=(0.5, 0.25, 0.25, 0))
print(score)